# High-Resolution EoS Curve Generator

## Detailed Analysis

This script is used to generate individual, high-fidelity Equation of State (EoS) curves. While the bulk worker scripts (`worker_hadronic_gen.py` and `worker_quark_gen.py`) focus on generating large datasets of macroscopic observables (Mass, Radius, Lambda) for machine learning, this script returns both the macroscopic sequence and the underlying microscopic $P$-$\epsilon$ grid.

It is primarily used by visualization scripts (such as `plot_theoretical_eos.py`) to render the "Theoretical Prior" bands, showing the full range of pressures and energy densities covered by the model parameter space. The physics logic for parameter sampling matches the bulk generators exactly to ensure the visualized curves are representative of the training data.

## Physics and Math

The script employs two distinct physical models depending on the requested mode.

### 1. Hadronic Scaling
To explore the parameter space efficiently, **homologous scaling** is applied to parent EoS models. If a parent model has a maximum mass $M_{ref}$, a scaled version with target mass $M_{target}$ is obtained via the scaling factor $\alpha$:

$$
\alpha = \left( \frac{M_{ref}}{M_{target}} \right)^2
$$

The pressure and energy density are transformed as:

$$
P'(r) = \alpha P(r), \quad \epsilon'(r) = \alpha \epsilon(r)
$$

### 2. Quark EoS (Algebraic)
For Quark stars, the Generalized CFL model relates pressure $P$ and energy density $\epsilon$ through the chemical potential $\mu$:

$$
P(\mu) = \frac{3}{4\pi^2}\mu^4 + \frac{3}{\pi^2}\left(\Delta^2 - \frac{m_s^2}{4}\right)\mu^2 - B
$$

$$
\epsilon(\mu) = \frac{3}{4\pi^2}\mu^4 + \frac{3}{\pi^2}\left(\Delta^2 - \frac{m_s^2}{4}\right)\mu^2 + B
$$

where $B$ is the Bag Constant, $\Delta$ is the gap energy, and $m_s$ is the strange quark mass.

## Code Walkthrough

### 1. Initialization and Configuration
The function accepts a `mode` ('hadronic' or 'quark'), a dictionary of `baselines`, and a random `seed`. Physics constants are loaded from the global configuration.

```python
np.random.seed(seed)
hc = CONSTANTS['HC']   
m_n = CONSTANTS['M_N'] 
```

### 2. Hadronic Generation Logic
If `mode == 'hadronic'`, the script follows the "Smart Inverse Sampling" strategy:
1.  Two parent models are selected from the library.
2.  A mixing weight $w$ is chosen.
3.  A scaling factor $\delta$ (and thus $\alpha$) is calculated to force the star's maximum mass into the target range (`H_M_MIN_TARGET` to `H_M_MAX_TARGET`).

Once the parameters are set, the TOV equations are solved. If the resulting star passes all filters (Causality, Radius Cap), the script proceeds to generate a **Dense EoS Grid**.

```python
# Dense grid for visualization
p_max = 3000.0 
p_grid = np.logspace(-4.0, np.log10(p_max), 100)

for p_val in p_grid:
    if p_val > p_trans_mix:
        # Core Region: Apply scaling and mixing
        p_base = p_val / alpha
        # ... (evaluate parent EoS) ...
        raw_e = ((vA**w) * (vB**(1.0-w))) * alpha
        eps_grid.append(raw_e)
    else:
        # Crust Region
        val = crust_funcs['c...'][0](p_val)
        eps_grid.append(val)
```
This loop ensures that a smooth, uniformly spaced $P$-$\epsilon$ curve is returned, which is necessary for clean plotting, as the adaptive ODE solver used in `solve_sequence` may produce irregular steps.

### 3. Quark Generation Logic
If `mode == 'quark'`, the script uses "Target-Driven Inverse Sampling":
1.  Microphysics parameters ($m_s$, $\Delta$) are sampled.
2.  The Stability Window ($B_{min}, B_{max}$) is calculated.
3.  The Bag Constant $B$ is inferred from a target maximum mass.

After validating the TOV solution, the Dense EoS Grid is generated by solving the algebraic CFL equations for each pressure in `p_grid`.

```python
coeff_c = -(p_val_geom + B_geom)
det = coeff_b**2 - 4*coeff_a*coeff_c
mu2 = (-coeff_b + np.sqrt(det)) / (2*coeff_a)
val_eps = val_eps_geom * hc
```
This explicitly maps the pressure range back to energy density, accounting for the vacuum pressure $B$.

## Visualization Output

This script does not produce a plot file directly. Instead, it returns the raw data `(curve, eos_arr)` used by **`src/visualize/plot_theoretical_eos.py`**.

*   **`curve`**: The Mass-Radius sequence (Macroscopic).
*   **`eos_arr`**: The Energy Density vs. Pressure array (Microscopic).

These outputs allow the visualization scripts to render the "theoretical bands" (the background regions in EoS plots) that represent the full prior parameter space explored by the generator.